In [32]:
import pandas as pd
import os
folders =['Finding Labels','Infiltration','Atelectasis','Effusion','Nodule','Pneumothorax','Mass','Consolidation','Pleural_Thickening','Cardiomegaly','Fibrosis','Edema','Multiple']

# values =df['Finding Labels'].value_counts() #Fileter just for PA view images (exclude AP)
# df[df['Finding Labels'].str.contains('^\|[\d|]*', regex=True)]
os.mkdir(f"{os.getcwd()}/dicom/NIH_images/PA")
os.mkdir(f"{os.getcwd()}/dicom/NIH_images/AP")
for folder in folders:
  os.mkdir(f"{os.getcwd()}/dicom/NIH_images/PA/{folder}")
  os.mkdir(f"{os.getcwd()}/dicom/NIH_images/AP/{folder}")


FileExistsError: [Errno 17] File exists: '/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/PA'

In [41]:
df = pd.read_csv('Data_Entry_2017_v2020.csv')
df[df['View Position']=='AP'].head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
12,00000004_000.png,Mass|Nodule,0,4,82,M,AP,2500,2048,0.168,0.168
14,00000005_001.png,No Finding,1,5,69,F,AP,2500,2048,0.168,0.168
15,00000005_002.png,No Finding,2,5,69,F,AP,2500,2048,0.168,0.168
33,00000011_005.png,Infiltration,5,11,75,M,AP,2500,2048,0.168,0.168
39,00000013_023.png,Infiltration|Mass|Pneumothorax,1,13,56,M,AP,2500,2048,0.168,0.168


In [39]:
import pathlib
list(pathlib.Path(f"{os.getcwd()}/dicom/NIH_images").glob('*.png'))

[PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00018805_001.png'),
 PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00005868_002.png'),
 PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00009663_000.png'),
 PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00022156_007.png'),
 PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00005395_004.png'),
 PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00006966_004.png'),
 PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00000324_013.png'),
 PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00007442_018.png'),
 PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00003923_012.png'),
 PosixPath('/Users/white/Documents/AlexProjects/DICOM_OCC_MED/dicom/NIH_images/00028483_007.png'),
 PosixPath